In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competicao-gthc-janeiro-2026/olist_customers_dataset.csv
/kaggle/input/competicao-gthc-janeiro-2026/olist_sellers_dataset.csv
/kaggle/input/competicao-gthc-janeiro-2026/olist_order_items_dataset.csv
/kaggle/input/competicao-gthc-janeiro-2026/olist_products_dataset.csv
/kaggle/input/competicao-gthc-janeiro-2026/olist_geolocation_dataset.csv
/kaggle/input/competicao-gthc-janeiro-2026/product_category_name_translation.csv
/kaggle/input/competicao-gthc-janeiro-2026/olist_order_payments_dataset.csv
/kaggle/input/competicao-gthc-janeiro-2026/TREINAMENTO/pedidos_treinamento.csv
/kaggle/input/competicao-gthc-janeiro-2026/TREINAMENTO/reviews_treinamento.csv
/kaggle/input/competicao-gthc-janeiro-2026/TESTE/pedidos_teste.csv


## IMPORTANDO DADOS E BIBLIOTECAS

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 50)             # permite a visualização de 50 colunas do dataframe
#pd.options.display.float_format = '{:.2f}'.format    # pandas: para todos os números aparecerem com duas casas decimais

In [3]:
customers = pd.read_csv("/kaggle/input/competicao-gthc-janeiro-2026/olist_customers_dataset.csv")
sellers = pd.read_csv("/kaggle/input/competicao-gthc-janeiro-2026/olist_sellers_dataset.csv")
order_items = pd.read_csv("/kaggle/input/competicao-gthc-janeiro-2026/olist_order_items_dataset.csv")
products = pd.read_csv("/kaggle/input/competicao-gthc-janeiro-2026/olist_products_dataset.csv")
geolocation = pd.read_csv("/kaggle/input/competicao-gthc-janeiro-2026/olist_geolocation_dataset.csv")
category_name_translation = pd.read_csv("/kaggle/input/competicao-gthc-janeiro-2026/product_category_name_translation.csv")
order_payments = pd.read_csv("/kaggle/input/competicao-gthc-janeiro-2026/olist_order_payments_dataset.csv")
pedidos_treinamento = pd.read_csv("/kaggle/input/competicao-gthc-janeiro-2026/TREINAMENTO/pedidos_treinamento.csv")
reviews_treinamento = pd.read_csv("/kaggle/input/competicao-gthc-janeiro-2026/TREINAMENTO/reviews_treinamento.csv")
pedidos_teste = pd.read_csv("/kaggle/input/competicao-gthc-janeiro-2026/TESTE/pedidos_teste.csv")

## Que comecem as análises

In [4]:
print(f"customers: {customers.shape}")
print(f"sellers: {sellers.shape}")
print(f"order_items: {order_items.shape}")
print(f"products: {products.shape}")
print(f"geolocation: {geolocation.shape}")
print(f"category_name_translation: {category_name_translation.shape}")
print(f"order_payments: {order_payments.shape}")
print(f"pedidos_treinamento: {pedidos_treinamento.shape}")
print(f"reviews_treinamento: {reviews_treinamento.shape}")
print(f"pedidos_teste: {pedidos_teste.shape}")

customers: (99441, 5)
sellers: (3095, 4)
order_items: (112650, 7)
products: (32951, 9)
geolocation: (1000163, 5)
category_name_translation: (71, 2)
order_payments: (103886, 5)
pedidos_treinamento: (74887, 8)
reviews_treinamento: (75385, 7)
pedidos_teste: (20943, 8)


### Tabela pedidos_treinamento (principal)

### Tabela reviews_treinamento (principal)

### Tabela customers

In [5]:
customers.dtypes

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

In [6]:
customers.isna().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [7]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


### Tabela sellers 

In [8]:
# THIS STUPID SHIT DIDN'T SAVE ANYTHING I DID I SPENT AN HOUR TRYING TO SAVE IT AAAAAAAAAAAAAAAAAAAAAAAAA

### Tabela order_items

### Tabela products

### Tabela geolocation

### Tabela category_name_translation

### Tabela order_payments

### Tabela pedidos_teste